<a href="https://colab.research.google.com/github/kinranlau/COMSOL_colloid_interaction/blob/main/SVM_prediction/%5BGUI%5D_Predict_colloid_interaction_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predict whether the interaction is favorable

**By fitting the simulated data with a SVM, this notebook provides an interactive GUI to predict whether a particular interaction is favorable (below or above kT) given a set of parameters.**

<br>

See the original paper:

*The Multivariate Interaction between Au and TiO<sub>2</sub> Colloids: The Role of Surface Potential, Concentration and Defects*

---
<p align="center">
  <img src="https://github.com/kinranlau/COMSOL_colloid_interaction/blob/main/SVM_prediction/Model%20configuration.gif?raw=true" width="600">
</p>

- this model considers the interaction between a **negative particle (e.g. Au)** and a **negative surface with positive/neutral defects (e.g. TiO<sub>2</sub>)** in water,
- with 116160 different combinations of these 7 parameters:
  - $V_{Part}:$ Particle potential
  - $V_{Surf}:$ Surface potential
  - $V_{Def}:$ Defect potential
  - $DD:$ Defect density
  - $Conc:$ Concentration
  - $R:$ Particle radius
  - $A_H:$ Hamaker constant

<br>

- the electrostatic interaction energy was computed with the software COMSOL
- and the van der Waals contribution was added subsequently

<br>

- this notebook takes the computed data and fit them with SVM (Support Vector Machine)
- so you can input a combination of these 7 parameters
- and predict whether the interaction is energetically feasible or not (below or above kT)


<br>

> This notebook is divided into various "*cells*", and you can run them one by one by clicking the "*Run cell*" button which looks like a play button.

In [1]:
#@title Import data and libraries { display-mode: "form" }

# import data
import pandas as pd
results = pd.read_csv('https://raw.githubusercontent.com/kinranlau/COMSOL_colloid_interaction/main/SVM_prediction/results_varyvdw_SVM.csv')

# select features and label
# split into training and test set
# scale the features
# run SVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import numpy as np

# first select the 7 features that we want:
# 1. colloid radius
# 2. V_particle
# 3. V_defect
# 4. V_surface
# 5. defect density
# 6. conc.
# 7. hamaker constant

features = results[['R/l_D',
                    'Particle potential',
                    'Defect potential',
                    'Surface potential',
                    'Defect density',
                    'Concentration / mM',
                    'Hamaker constant / 10^-21 J']]

# our label is the whether the final energy is "above/below kT"
# add another column to show below/above kT
# 1: below kT; 0: above kT
below_kT = lambda x: 1 if x <= 1.001 else 0
results['below kT'] = results['Energy / kT'].apply(below_kT)

label = results['below kT']

In [2]:
#@title Split data into training and test set (80:20) { display-mode: "form" }

# split into training and test set
training_data, validation_data, training_labels, validation_labels = train_test_split(features, label, test_size = 0.2)

# scale the feature data so it has mean = 0 and standard deviation = 1
scaler = StandardScaler()
training_data = scaler.fit_transform(training_data)
validation_data = scaler.fit_transform(validation_data)

In [3]:
#@title Fit data with SVM { display-mode: "form" }
#@markdown This might take ~30 seconds to run.

#@markdown You should get an accuracy of about 97%.

# fit with SVM
# using default C and gamma
SVM = SVC()
SVM.fit(training_data, training_labels)

# score the SVM model
SVM_score = SVM.score(validation_data, validation_labels)
print(f'The model has an accuracy of {SVM_score*100:.2f}%.')

The model has an accuracy of 97.99%.


In [4]:
#@title Helper functions for predicting "below kT" or "above kT" { display-mode: "form" }

# predict below kT or not with SVM
def predict_below_kT(para):
    # predict "below kT" or not
    # 1: below kT; 0: above kT

    # scale input
    para = scaler.transform(para)

    # predict by SVM
    SVM_pred = SVM.predict(para)[0]

    # decision function:
    # larger absolute value = higher confidence
    # close to zero = very low confidence
    SVM_decision_func = SVM.decision_function(para)[0]

    if SVM_pred == 0:
      print(f'Prediction by SVM: Above kT! The decision function is {SVM_decision_func:.3f}.')
    else:
      print(f'Prediction by SVM: Below kT! The decision function is {SVM_decision_func:.3f}.')
##################################################################
# select the unique values of the 7 features we have
# 1. colloid radius
# 2. V_particle
# 3. V_defect
# 4. V_surface
# 5. defect density
# 6. conc.
# 7. Hamaker constant

colloid_radius_unique = sorted(results['R/l_D'].unique())
V_particle_unique =  sorted(results['Particle potential'].unique())
V_defect_unique =  sorted(results['Defect potential'].unique())

# select results with defects (DD!=0)
results_def = results[results['Defect density'] != 0]
# unique values (4) for defective surface
V_surface_def_unique =  sorted(results_def['Surface potential'].unique())
# unique values (21) for perfect surface
V_surface_unique =  sorted(results['Surface potential'].unique())

DD_unique = sorted(results['Defect density'].unique())
conc_unique = sorted(results['Concentration / mM'].unique())
hamaker_unique = sorted(results['Hamaker constant / 10^-21 J'].unique())
##################################################################
# Compare with original dataset to look for the closest match
def compare_data(radius, V_particle, V_defect, V_surface, DD, conc, hamaker):
    # find the closest match by comparing to the unique values
    radius_close = min(colloid_radius_unique, key=lambda x:abs(x-radius))
    V_particle_close = min(V_particle_unique, key=lambda x:abs(x-V_particle))
    V_defect_close = min(V_defect_unique, key=lambda x:abs(x-V_defect))

    # compare with different unique values depending on perfect or defective surfaces
    if DD == 0.0:
      V_surface_close = min(V_surface_unique, key=lambda x:abs(x-V_surface))
    else:
      V_surface_close = min(V_surface_def_unique, key=lambda x:abs(x-V_surface))

    DD_close = min(DD_unique, key=lambda x:abs(x-DD))
    conc_close = min(conc_unique, key=lambda x:abs(x-conc))
    hamaker_close = min(hamaker_unique, key=lambda x:abs(x-hamaker))

    # sanity check: if DD is 0, then V_defect should also be set to zero
    if DD_close == 0.0:
      V_defect_close = 0.0

    # select the closest match from the original data
    df = results[\
    (results['R/l_D'] == radius_close) & \
    (results['Particle potential'] == V_particle_close) & \
    (results['Defect potential'] == V_defect_close) & \
    (results['Surface potential'] == V_surface_close) & \
    (results['Defect density'] == DD_close) & \
    (results['Concentration / mM'] == conc_close) & \
    (results['Hamaker constant / 10^-21 J'] == hamaker_close)
    ]

    # read is it below or above kT
    # 1: below kT; 0: above kT
    below_kT = df['below kT'].values[0]

    # convert the closest match to readable input
    # 1. colloid radius (from l_D to nm, l_D = 0.5 nm)
    radius_report = radius_close * 0.5
    # 2. V_particle (from dimensionless potential to mV)
    V_particle_report = V_particle_close * 25.7
    # 3. V_defect (from dimensionless potential to mV)
    V_defect_report = V_defect_close * 25.7
    # 4. V_surface (from dimensionless potential to mV)
    V_surface_report = V_surface_close * 25.7
    # 5. defect density (no conversion needed)
    DD_report = DD_close
    # 6. conc. (mM, no conversion needed)
    conc_report = conc_close
    # 7. Hamaker constant (zJ, no conversion needed)
    hamaker_report = hamaker_close

    # print results
    print(f'The closest match in the dataset: \nV_Part: {V_particle_report:.1f} mV \nV_Surf: {V_surface_report:.1f} mV \nV_Def: {V_defect_report:.1f} mV \nDD: {DD_report} \nConc: {conc_report} mM \nR: {radius_report} nm \nA_H: {hamaker_report} zJ')
    print('Below kT' if below_kT == 1 else 'Above kT')

In [21]:
#@markdown <img src="https://github.com/kinranlau/COMSOL_colloid_interaction/blob/main/SVM_prediction/Parameters.png?raw=true" width="400">

#@title Input your parameters and run the prediction!  { display-mode: "form" }
#@markdown <br>

#@markdown ### 1. $V_{Part}:$ Particle potential (mV or dimensionless potential)
#@markdown - You can select the unit of the potentials ("mV" or "Dimensionless potential").
#@markdown - At 298 K, dimensionless potential can be converted to mV by simply multiplying by 25.7.
unit_of_potential = 'mV' #@param ["mV", "Dimensionless potential"] {allow-input: false}
V_particle = -35 #@param {type:"number"}
#@markdown ---

#@markdown ### 2. $V_{Surf}:$ Surface potential (mV or dimensionless potential)
V_surface =  -35#@param {type:"number"}
#@markdown ---

#@markdown ### 3. $V_{Def}:$ Defect potential (mV or dimensionless potential)
V_defect =  0#@param {type:"number"}
#@markdown ---

#@markdown ### 4. $DD:$ Defect density
#@markdown - Defects are arranged in a cubic primitive array, and defect density is defined by $\frac{\pi {l_D}^{2}}{a^2}$.
#@markdown - $l_D$ is 0.5 nm in our model.

#@markdown <img src="https://github.com/kinranlau/COMSOL_colloid_interaction/blob/main/SVM_prediction/Defect%20definition.png?raw=true" width="500">
DD =  0#@param {type:"number"}
#@markdown ---

#@markdown ### 5. $Conc:$ Concentration of salt (mM)
conc =  0.5#@param {type:"number"}
#@markdown ---

#@markdown ### 6. $R:$ Particle radius (nm)
radius =  2.5#@param {type:"number"}
#@markdown ---

#@markdown ### 7. $A_H:$ Hamaker constant (zJ)
#@markdown - TiO<sub>2</sub> - TiO<sub>2</sub>: 50 - 60 zJ
#@markdown - Au - Au: 90 - 300 zJ
#@markdown - The Hamaker constant for Au - TiO<sub>2</sub> can be estimated by taking the harmonic or geometric mean (100 zJ was used in the paper).
# TiO2 - TiO2: ~50-60
# Au - Au: ~150
hamaker =  100#@param {type:"number"}
#@markdown ---
############################################################
# conversion of units if needed
# the model fitted the potentials in dimensionless potential
# so if the input is in mV, then conversion to dimensionless potential is needed
if unit_of_potential == 'mV':
  V_particle /= 25.7
  V_surface /= 25.7
  V_defect /= 25.7

# in our model, the particle radius is defined relative to l_D which is 0.5 nm
# so we need to convert nm back to units of l_D
radius /= 0.5
############################################################
# concat parameters
para_array = np.array([radius, V_particle, V_defect, V_surface, DD, conc, hamaker])
para_array = para_array.reshape(1,7)

para_label = ['R/l_D',
              'Particle potential',
              'Defect potential',
              'Surface potential',
              'Defect density',
              'Concentration / mM',
              'Hamaker constant / 10^-21 J']

para = pd.DataFrame(para_array, columns =  para_label)

# predict above/below kT with probability
predict_below_kT(para)
print('Decision function: a larger absolute value means higher confidence; closer to zero means less confidence.')
############################################################
# compare with original dataset to find the closest match
print('\n')
compare_data(radius, V_particle, V_defect, V_surface, DD, conc, hamaker)

Prediction by SVM: Above kT! The decision function is -1.382.
Decision function: a larger absolute value means higher confidence; closer to zero means less confidence.


The closest match in the dataset: 
V_Part: -38.5 mV 
V_Surf: -36.0 mV 
V_Def: 0.0 mV 
DD: 0.0 
Conc: 0.5 mM 
R: 2.5 nm 
A_H: 100.0 zJ
Above kT
